In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Use to reset TF Graphs
tf.reset_default_graph()

Extracting images out of ZIP file

In [0]:
from zipfile import ZipFile
with ZipFile('/content/gdrive/My Drive/test.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
  
with ZipFile('/content/gdrive/My Drive/train.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [0]:
classes_train = sorted(list(filter(lambda x: os.path.isdir('/content/train/' + x), os.listdir('/content/train'))))

Splitting data into Train and Validation Set

In [0]:
!pip install split_folders


In [0]:
import split_folders
split_folders.ratio('/content/train', output="output", seed=1337, ratio=(.8, .2)) # default values

Copying files: 44016 files [00:07, 5591.64 files/s]


Generating Image Generators for train and validation

In [0]:
batch_size = 128
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        '/content/output/train', 
        target_size=(200, 200),  
        batch_size=batch_size,
        classes = classes_train,
        class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1/255)
val_generator = val_datagen.flow_from_directory(
        '/content/output/val', 
        target_size=(200, 200),  
        batch_size=batch_size,
        classes = classes_train,
        class_mode='categorical')

Found 35196 images belonging to 39 classes.
Found 8820 images belonging to 39 classes.


Model Creation 



In [0]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x 200 with 3 bytes color
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 5 output neurons for 5 classes with the softmax activation
    tf.keras.layers.Dense(39, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 198, 198, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 97, 97, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        1

Compiling Model before training and testing

In [0]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0005),
              metrics=['acc'])

Creating callbacks and train

In [0]:
total_sample=train_generator.n
n_epochs = 30

checkpoint = keras.callbacks.ModelCheckpoint('/content/abc.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tb = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')


history = model.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        validation_data = val_generator,
        callbacks = [checkpoint, tb],
        verbose=1)

Saving model other than checkpoint

In [0]:
from tensorflow import keras
keras.models.save_model(model, "model.h5")

Loading model back from the file ( saved as checkpoint from callback )

In [0]:
model = tf.keras.models.load_model("/content/gdrive/My Drive/abc.h5")
model.evaluate_generator(val_generator)

[0.18151555059180743, 0.95011336]

Predicting labels for Testing Data and converting in JSON

In [0]:
test_images = [f for f in os.listdir('/content/test_imgs')]
dataset = np.ndarray(shape=(len(test_images), 200, 200, 3))

import cv2
for i, file_name in enumerate(test_images):
  try:
    img = cv2.imread('/content/test_imgs/'+file_name)
    img_resize = cv2.resize(img, (200,200))
    dataset[i] = np.array(img_resize)/255.0
  except:
    print(file_name)
dataset.shape

(11004, 200, 200, 3)

In [0]:
results = model.predict_classes(dataset)

final_json = {}
for file_name, result in zip(test_images, results):
  final_json[file_name] = classes_train[result]
  
  
import json
with open('prediction.json', 'w') as outfile:
    json.dump(final_json, outfile)

Downloading the JSON File

In [0]:
from google.colab import files
files.download('predictions.json')

FileNotFoundError: ignored

Converting and saving as TFLite model

In [0]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model_file('abc.h5')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open("abc_opt.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 34 variables.
INFO:tensorflow:Converted 34 variables to const ops.


244376

Downloading TFLite Model

In [0]:
from google.colab import files
files.download('abc_opt.tflite')

Loading Tensorflow Lite model back and feeding the validation data

In [0]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="abc_opt.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# input_shape = input_details[0]['shape']
# interpreter.set_tensor(input_details[0]['index'], X[0].reshape(1,200,200,3))

# interpreter.invoke()

# output_data = interpreter.get_tensor(output_details[0]['index'])
# print(output_data)


Comparing the validation set with the TF model and TF_Lite model

In [0]:
correct_count, lite_correct_count, incorrect_count, lite_incorrect_count = 0, 0, 0, 0
while True:
  try:
    print("New Batch")
    X, y = val_generator.next()
    for i in range(len(X)):
      interpreter.set_tensor(input_details[0]['index'], X[i].reshape(1,200,200,3))
      interpreter.invoke()
      output_data = interpreter.get_tensor(output_details[0]['index'])
      lite_pred_class = np.argmax(output_data)
      pred_class = np.argmax(model.predict(X[i].reshape(1,200,200,3)))
      orig_class = np.argmax(y[i])
      if pred_class == orig_class:
        correct_count = correct_count + 1
      if lite_pred_class == orig_class:
        lite_correct_count = lite_correct_count + 1
      if pred_class != orig_class:
        incorrect_count = incorrect_count + 1
      if lite_pred_class != orig_class:
        lite_incorrect_count = lite_incorrect_count + 1
    print("{} incorrectly identified by tflite".format(lite_incorrect_count))
    print("{} incorrectly identified by tf".format(incorrect_count))
  except Exception as e:
    print(e)
    break
    